In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC,  LinearSVC
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.decomposition import PCA
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score, confusion_matrix, classification_report, precision_score
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.datasets import make_classification
import xgboost as xgb
from classification_model import xgb_classification, lgb_classification_2, svm_classification_2, calculate_precision
rus = RandomUnderSampler(random_state=2021)

In [69]:
df = []
files = os.listdir('../res2')
for i in files:
    if not i.endswith('txt'):
        df.append(pd.read_csv('../res2/'+i))
df = pd.concat(df)
df = df[df.columns[2:]]
df.shape

(9617, 2081)

In [71]:
33 * 63 + 2 - 62

2019

In [49]:
df.head()

,no,axis_x,axis_y,girth,girth_full,area,elongation,eccentricity,convexity,recty,...,hu_m5_41,hu_m6_41,contrast_41,dissimilarity_41,homogeneity_41,energy_41,correlation_41,ASM_41,dense_41,id
0,20_35_-1,106.000000,133.500000,85.426406,92.426406,544.0,-0.484173,0.738936,1.190136,0.725333,...,3.008560e-07,-2.659284e-11,9.998720,0.081209,0.999138,0.999136,0.655664,0.998273,406.223721,0
1,20_35_-1,104.500000,134.500000,86.183765,88.183765,539.5,-0.246404,0.596910,1.179355,0.799259,...,3.322701e-06,2.105517e-09,4.995576,0.030827,0.999785,0.999785,0.163664,0.999571,398.305919,1
2,20_35_-1,103.307701,136.461533,82.840619,85.840619,507.5,-0.230711,0.582908,1.201928,0.770016,...,-1.920488e-08,1.756677e-11,4.227566,0.045268,0.999267,0.999256,0.409500,0.998513,391.780750,2
3,20_35_-1,104.692307,138.038452,85.083260,86.083260,500.0,-0.220799,0.573600,1.179209,0.784834,...,-4.677410e-08,-7.630613e-12,1.905849,0.035555,0.998969,0.998940,0.577581,0.997880,390.584196,3
4,20_35_-1,105.729500,137.475403,87.325901,92.325901,587.5,-0.159604,0.506290,1.178774,0.797346,...,-4.361854e-08,1.109048e-11,2.058403,0.038015,0.998925,0.998879,0.574417,0.997759,384.267989,4


In [72]:
def  calculate_shift(x):
    axis_cols = [('axis_x_{}'.format(i), 'axis_y_{}'.format(i)) for i in range(62)]
    shifts = []
    a, b = x['axis_x'], x['axis_y']
    p_a, p_b = a, b
    for i in axis_cols:
        a_hat, b_hat = x[i[0]], x[i[1]]
        shift = np.sqrt(np.power(a-a_hat, 2) + np.power(b-b_hat, 2))
        a, b = a_hat, b_hat
        shifts.append(shift)
    a_hat, b_hat = x[axis_cols[-1][0]], x[axis_cols[-1][1]]
    shift = np.sqrt(np.power(p_a-a_hat, 2) + np.power(p_b-b_hat, 2))
    sum = np.sum(shifts)
    shifts.append(shift)
    shifts.append(sum)
    shifts.append(x['id'])
    return shifts

In [73]:
# 计算位移
df['id'] = list(range(df.shape[0]))
res = df.apply(calculate_shift, axis=1)
shift_cols = ['shift_{}'.format(i) for i in range(62)]
shift_cols = shift_cols + ['shift_sum', 'distance', 'id']
shift = pd.DataFrame(data=list(res), columns=shift_cols)
X = pd.merge(df, shift, on='id')
X.drop(columns=['id'], inplace=True)

In [75]:
X[[i for i in X.columns if not i.startswith('axis')]].head()

,no,girth,girth_full,area,elongation,eccentricity,convexity,recty,roundness,major_axis,...,shift_54,shift_55,shift_56,shift_57,shift_58,shift_59,shift_60,shift_61,shift_sum,distance
0,925_19_-1,56.041630,60.041630,211.5,-0.456462,0.727040,1.165118,0.692744,0.737251,21.489479,...,9.851911,5.059226,8.253812,0.622414,0.897364,0.790720,1.476154,4.047068,43.807837,114.897199
1,925_19_-1,51.627416,58.627416,217.0,-0.264703,0.612204,1.159867,0.753472,0.793356,19.803518,...,8.253812,0.622414,0.897364,0.790720,1.476154,4.047068,2.236068,0.000000,42.722945,115.679515
2,925_19_-1,56.870057,60.870057,234.5,-0.231336,0.583482,1.184489,0.727357,0.795326,19.874460,...,0.897364,0.790720,1.476154,4.047068,2.236068,0.000000,0.500000,0.000000,43.410832,113.236375
3,925_19_-1,53.698484,57.698484,216.5,-0.199353,0.552093,1.193634,0.731419,0.817220,18.849867,...,1.476154,4.047068,2.236068,0.000000,0.500000,0.000000,1.500000,1.085302,43.726212,113.389904
4,925_19_-1,51.870057,56.870057,217.5,-0.289176,0.631115,1.195708,0.752595,0.845088,19.654890,...,2.236068,0.000000,0.500000,0.000000,1.500000,1.085302,0.115884,1.617682,41.419601,112.277453


In [74]:
X.to_csv('res/final_63.csv')

In [54]:
origin = pd.read_csv('../data/origin.csv')
origin = origin[origin.columns[1:]]

In [55]:
origin.head()

,no,win_no,start,end,parent,frame_no,axis_x,axis_y,girth,girth_full,...,hu_m6,contrast,dissimilarity,homogeneity,energy,correlation,ASM,label,dense,len_trace
0,96_21_-1,21,11,16,0_21_-1,11,385.500000,571.000000,29.656854,35.656854,...,3.815276e-11,3.053937,0.027140,0.999724,0.999723,0.273471,0.999446,0,307.035862,6
1,96_21_-1,21,11,16,0_21_-1,12,384.500000,569.000000,36.899495,37.899495,...,2.459672e-10,3.597688,0.029776,0.999718,0.999717,0.294037,0.999435,0,299.315254,6
2,96_21_-1,21,11,16,0_21_-1,13,385.000000,569.500000,35.727922,36.727922,...,2.901733e-08,3.748888,0.029114,0.999739,0.999739,0.251333,0.999478,0,297.527692,6
3,96_21_-1,21,11,16,0_21_-1,14,384.774353,570.969421,28.899495,29.899495,...,2.179500e-07,3.988327,0.025613,0.999816,0.999816,0.123756,0.999632,0,303.486928,6
4,96_21_-1,21,11,16,0_21_-1,15,385.300018,570.100037,30.727922,34.970562,...,-6.243646e-10,4.115033,0.028699,0.999764,0.999764,0.297897,0.999528,0,298.393304,6


In [56]:
41 - 5 - 2

34

In [61]:
33 * 43 + 2 - 42

1379

In [59]:
for i in df.columns:
    print(i)

no
axis_x
axis_y
girth
girth_full
area
elongation
eccentricity
convexity
recty
roundness
major_axis
minor_axis
cent_m0
cent_m1
cent_m2
cent_m3
cent_m4
cent_m5
cent_m6
hu_m0
hu_m1
hu_m2
hu_m3
hu_m4
hu_m5
hu_m6
contrast
dissimilarity
homogeneity
energy
correlation
ASM
label
dense
axis_x_0
axis_y_0
girth_0
girth_full_0
area_0
elongation_0
eccentricity_0
convexity_0
recty_0
roundness_0
major_axis_0
minor_axis_0
cent_m0_0
cent_m1_0
cent_m2_0
cent_m3_0
cent_m4_0
cent_m5_0
cent_m6_0
hu_m0_0
hu_m1_0
hu_m2_0
hu_m3_0
hu_m4_0
hu_m5_0
hu_m6_0
contrast_0
dissimilarity_0
homogeneity_0
energy_0
correlation_0
ASM_0
dense_0
axis_x_1
axis_y_1
girth_1
girth_full_1
area_1
elongation_1
eccentricity_1
convexity_1
recty_1
roundness_1
major_axis_1
minor_axis_1
cent_m0_1
cent_m1_1
cent_m2_1
cent_m3_1
cent_m4_1
cent_m5_1
cent_m6_1
hu_m0_1
hu_m1_1
hu_m2_1
hu_m3_1
hu_m4_1
hu_m5_1
hu_m6_1
contrast_1
dissimilarity_1
homogeneity_1
energy_1
correlation_1
ASM_1
dense_1
axis_x_2
axis_y_2
girth_2
girth_full_2
area_2
elo

In [ ]:
pri